In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from adjustText import adjust_text

# Read CSV files
lgb_metrics = pd.read_csv("lgbmedian_metrics.csv")
rf_metrics = pd.read_csv("rfmedian_metrics.csv")

# Add a column to indicate which model the data comes from
lgb_metrics["model"] = "LGB"
rf_metrics["model"] = "RF"

# Concatenate the two tables vertically
combined_metrics = pd.concat([rf_metrics, lgb_metrics], ignore_index=True)

# Rearrange column order
combined_metrics = combined_metrics[
    ["sampling_method", "dataset_level", "test_accuracy", "test_f1", "roc_auc", "model"]
]

# Calculate the difference and relative percentage of each metric compared to RF+SRS at the same dataset level
for metric in ["test_accuracy", "test_f1", "roc_auc"]:
    srs_rf_values = (
        combined_metrics[
            (combined_metrics["sampling_method"] == "SRS")
            & (combined_metrics["model"] == "RF")
        ][["dataset_level", metric]]
        .set_index("dataset_level")
        .to_dict()[metric]
    )

    combined_metrics[f"{metric}_diff"] = combined_metrics.apply(
        lambda row: row[metric] - srs_rf_values[row["dataset_level"]],
        axis=1,
    )
    combined_metrics[f"{metric}_pct_diff"] = combined_metrics.apply(
        lambda row: (row[metric] - srs_rf_values[row["dataset_level"]])
        / srs_rf_values[row["dataset_level"]]
        * 100,
        axis=1,
    )

# Create a new column combining sampling_method and model, with a line break after the model name
combined_metrics["sampling_model"] = (
    combined_metrics["model"] + "\n" + combined_metrics["sampling_method"]
)

# Remove the RF + SRS combination
combined_metrics = combined_metrics[
    ~(
        (combined_metrics["sampling_method"] == "SRS")
        & (combined_metrics["model"] == "RF")
    )
]

# Specify the X-axis labels and corresponding sampling methods from left to right
sampling_methods = [
    "BalancedSampling",
    "clhs",
    "FCMtp1sampled",
    "FCMtp2sampled",
    "FSCS",
    "kmeans",
    "SRS",
]

# Corresponding X-axis labels
sampling_labels = [
    "Balanced\nSampling",
    "CLHS",
    "FCM\nClu = class",
    "FCM\nClu = level",
    "FSCS",
    "K-means\nClu = class",
    "SRS",
]

# Generate new X-axis label order
sampling_model_order = []
sampling_label_new = []
for method, label in zip(sampling_methods, sampling_labels):
    sampling_model_order.append(f"RF\n{method}")
    sampling_model_order.append(f"LGB\n{method}")

    sampling_label_new.append(f"RF\n{label}")
    sampling_label_new.append(f"LGB\n{label}")

# Generate new percentage label order
sampling_modeldatalevel_order = []
for method, label in zip(sampling_methods, sampling_labels):
    for level in sorted(
        combined_metrics["dataset_level"].unique(),
        key=lambda x: [1000, 5000, 10000, 20000].index(x),
    ):
        if method != "SRS":
            sampling_modeldatalevel_order.append((f"RF\n{method}", level))
    for level in sorted(
        combined_metrics["dataset_level"].unique(),
        key=lambda x: [1000, 5000, 10000, 20000].index(x),
    ):
        sampling_modeldatalevel_order.append((f"LGB\n{method}", level))

# Remove the RF + SRS label
sampling_model_order.remove("RF\nSRS")
sampling_label_new.remove("RF\nSRS")

# Set font size
plt.rcParams.update({"font.size": 14})

# Visualization
metrics = ["test_accuracy", "test_f1", "roc_auc"]
metric_titles = ["Accuracy", "F1 Score", "ROC-AUC"]

# Visualize the metric improvement from RF+SRS to LGB/RF + each sampling method
fig, axes = plt.subplots(3, 1, figsize=(15, 22), sharey=False)
for ax, metric, title in zip(axes, metrics, metric_titles):

    bars = sns.barplot(
        x="sampling_model",
        y=f"{metric}_diff",
        hue="dataset_level",
        data=combined_metrics,
        order=sampling_model_order,  # Set X-axis order
        ax=ax,
    )
    ax.set_title(
        f"Difference in {title} from RF+SRS to LGB/RF + Sampling Methods", fontsize=16
    )
    ax.set_xlabel("Sampling Method + Model", fontsize=15, labelpad=10)
    ax.set_ylabel(f"Difference in {title}", fontsize=15, labelpad=10)
    ax.legend(title="Dataset Level", loc="lower left", bbox_to_anchor=(0, 0))
    ax.axhline(0, color="gray", linestyle="--")
    ax.set_xticks(range(len(sampling_model_order)))
    ax.set_xticklabels(sampling_label_new, fontsize=12)
    ax.grid(True, which="major", axis="y", linestyle="-", linewidth=0.6)
    ax.grid(True, which="minor", axis="y", linestyle="--", linewidth=0.25)
    ax.yaxis.set_major_locator(plt.MultipleLocator(0.05))
    ax.yaxis.set_minor_locator(plt.MultipleLocator(0.01))

    # Add the current value at the end of each bar
    texts = []
    for p in bars.patches:
        height = p.get_height()
        diff_label = format(height, ".2f")

        # Find the corresponding {metric}_diff position
        matching_row = combined_metrics[combined_metrics[f"{metric}_diff"] == height]

        if not matching_row.empty:
            pct_label = format(matching_row[f"{metric}_pct_diff"].values[0], ".1f")
        else:
            pct_label = "N/A"

        if diff_label != "0.00" and diff_label != "-0.00":
            left = p.get_x() + p.get_width() / 2.0
            if height > 0:
                top = height
                text = ax.text(
                    left,
                    top * 0.7,
                    f"{diff_label}\n{pct_label}%",
                    ha="center",
                    va="bottom",
                    fontsize=11,
                )
                texts.append(text)
            else:
                top = height
                text = ax.text(
                    left,
                    top * 0.7,
                    f"{diff_label}\n{pct_label}%",
                    ha="center",
                    va="top",
                    fontsize=11,
                )
                texts.append(text)

    adjust_text(texts, add_objects=bars, only_move="y", ax=ax, max_move=(4, 4))

    ax.axvspan(
        sampling_model_order.index("LGB\nSRS") - 0.5,
        sampling_model_order.index("LGB\nSRS") + 0.5,
        facecolor="lightgray",
        edgecolor="black",
        alpha=0.15,
    )

plt.tight_layout()
plt.show()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from adjustText import adjust_text

# Read CSV files
lgb_metrics = pd.read_csv("lgbKsatmedian_metrics.csv")
rf_metrics = pd.read_csv("rfKsatmedian_metrics.csv")

# Add a column to indicate which model the data comes from
lgb_metrics["model"] = "LGB"
rf_metrics["model"] = "RF"

# Concatenate the two tables vertically
combined_metrics = pd.concat([rf_metrics, lgb_metrics], ignore_index=True)

# Rearrange column order
combined_metrics = combined_metrics[
    ["sampling_method", "dataset_level", "r2", "rmsle", "model"]
]

# Calculate the difference and relative percentage of each metric compared to RF+SRS at the same dataset level
for metric in ["r2", "rmsle"]:
    srs_rf_values = (
        combined_metrics[
            (combined_metrics["sampling_method"] == "SRS")
            & (combined_metrics["model"] == "RF")
        ][["dataset_level", metric]]
        .set_index("dataset_level")
        .to_dict()[metric]
    )

    combined_metrics[f"{metric}_diff"] = combined_metrics.apply(
        lambda row: row[metric] - srs_rf_values[row["dataset_level"]],
        axis=1,
    )
    combined_metrics[f"{metric}_pct_diff"] = combined_metrics.apply(
        lambda row: (row[metric] - srs_rf_values[row["dataset_level"]])
        / srs_rf_values[row["dataset_level"]]
        * 100,
        axis=1,
    )

# Create a new column combining sampling_method and model, with a line break after the model name
combined_metrics["sampling_model"] = (
    combined_metrics["model"] + "\n" + combined_metrics["sampling_method"]
)

# Remove the RF + SRS combination
combined_metrics = combined_metrics[
    ~(
        (combined_metrics["sampling_method"] == "SRS")
        & (combined_metrics["model"] == "RF")
    )
]

# Specify the X-axis labels and corresponding sampling methods from left to right
sampling_methods = [
    "BalancedSampling",
    "clhs",
    "FCMtp2sampled",
    "FSCS",
    "SRS",
]

# Corresponding X-axis labels
sampling_labels = [
    "Balanced\nSampling",
    "CLHS",
    "FCM\nClu = level",
    "FSCS",
    "SRS",
]

# Generate new X-axis label order
sampling_model_order = []
sampling_label_new = []
for method, label in zip(sampling_methods, sampling_labels):
    sampling_model_order.append(f"RF\n{method}")
    sampling_model_order.append(f"LGB\n{method}")

    sampling_label_new.append(f"RF\n{label}")
    sampling_label_new.append(f"LGB\n{label}")

# Generate new percentage label order
sampling_modeldatalevel_order = []
for method, label in zip(sampling_methods, sampling_labels):
    for level in sorted(
        combined_metrics["dataset_level"].unique(),
        key=lambda x: [1000, 5000, 10000].index(x),
    ):
        if method != "SRS":
            sampling_modeldatalevel_order.append((f"RF\n{method}", level))
    for level in sorted(
        combined_metrics["dataset_level"].unique(),
        key=lambda x: [1000, 5000, 10000].index(x),
    ):
        sampling_modeldatalevel_order.append((f"LGB\n{method}", level))

# Remove the RF + SRS label
sampling_model_order.remove("RF\nSRS")
sampling_label_new.remove("RF\nSRS")

# Set font size
plt.rcParams.update({"font.size": 14})

# Visualization
metrics = ["r2", "rmsle"]
metric_titles = ["R2", "RMSLE"]

# Visualize the metric improvement from RF+SRS to LGB/RF + each sampling method
fig, axes = plt.subplots(2, 1, figsize=(15, 22), sharey=False)
for ax, metric, title in zip(axes, metrics, metric_titles):

    bars = sns.barplot(
        x="sampling_model",
        y=f"{metric}_diff",
        hue="dataset_level",
        data=combined_metrics,
        order=sampling_model_order,  # Set X-axis order
        ax=ax,
    )
    ax.set_title(
        f"Difference in {title} from RF+SRS to LGB/RF + Sampling Methods", fontsize=16
    )
    ax.set_xlabel("Sampling Method + Model", fontsize=15, labelpad=10)
    ax.set_ylabel(f"Difference in {title}", fontsize=15, labelpad=10)
    ax.legend(title="Dataset Level", loc="lower left", bbox_to_anchor=(0, 0))
    ax.axhline(0, color="gray", linestyle="--")
    ax.set_xticks(range(len(sampling_model_order)))
    ax.set_xticklabels(sampling_label_new, fontsize=15)
    ax.grid(True, which="major", axis="y", linestyle="-", linewidth=0.6)
    ax.grid(True, which="minor", axis="y", linestyle="--", linewidth=0.25)
    ax.yaxis.set_major_locator(plt.MultipleLocator(0.05))
    ax.yaxis.set_minor_locator(plt.MultipleLocator(0.01))

    # Add the current value at the end of each bar

    texts = []
    for p in bars.patches:
        height = p.get_height()
        diff_label = format(height, ".2f")

        # Find the corresponding {metric}_diff position
        matching_row = combined_metrics[combined_metrics[f"{metric}_diff"] == height]

        if not matching_row.empty:
            pct_label = format(matching_row[f"{metric}_pct_diff"].values[0], ".1f")
        else:
            pct_label = "N/A"

        if diff_label != "0.00" and diff_label != "-0.00":
            left = p.get_x() + p.get_width() / 2.0
            if height > 0:
                top = height
                text = ax.text(
                    left,
                    top,
                    f"{diff_label}\n{pct_label}%",
                    ha="center",
                    va="bottom",
                    fontsize=11,
                )
                texts.append(text)
            else:
                top = height
                text = ax.text(
                    left,
                    top,
                    f"{diff_label}\n{pct_label}%",
                    ha="center",
                    va="top",
                    fontsize=11,
                )
                texts.append(text)

    adjust_text(texts, add_objects=bars, only_move="y+", ax=ax, max_move=(0.1, 0.1))

    if metric == "rmsle":
        ax.invert_yaxis()  # Invert y-axis

    ax.axvspan(
        sampling_model_order.index("LGB\nSRS") - 0.5,
        sampling_model_order.index("LGB\nSRS") + 0.5,
        facecolor="lightgray",
        edgecolor="black",
        alpha=0.15,
    )

plt.tight_layout()
plt.savefig("Ksatmetric.jpg", format="jpg", dpi=800, bbox_inches="tight")
plt.show()

From the cell below, it can be run independently.

In [ ]:
# Data loading and train/test set construction

import pandas as pd
import numpy as np
import os
from sklearn.utils import resample

# Read the original dataset
class_df = pd.read_csv("sampleddata/class_df.csv", index_col=0)

# Define sample sizes and sampling methods
sample_levels = [1000, 5000, 10000, 20000]

sampling_methods = [
    "BalancedSampling_sampled_data",
    "clhs_sampled_data",
    "FCMtp1sampled_data",
    "FCMtp2sampled_data",
    "FSCS_sampled_data",
    "kmeans_sampled_data",
]

# Define covariates and target variable
covariates = [
    "elevation",
    "aspect",
    "slope",
    "horizontal_distance_to_hydrology",
    "Vertical_Distance_To_Hydrology",
    "Horizontal_Distance_To_Roadways",
    "Hillshade_9am",
    "Hillshade_Noon",
    "Hillshade_3pm",
    "Horizontal_Distance_To_Fire_Points",
]
target = "class"

# Store train and test sets
train_test_data = {}

# Loop through each sample size and sampling method
for level in sample_levels:
    for t in range(1, 21):
        for method in sampling_methods:
            sample_file = f"sampleddata/combined_samples/{method}_{level}_set_{t}.csv"

            if not os.path.exists(sample_file):
                print(f"File {sample_file} does not exist. Skipping...")
                continue

            sample_data = pd.read_csv(sample_file, index_col=0)
            X_train = sample_data[covariates]
            y_train = sample_data[target]
            train_indices = sample_data.index

            # Generate the corresponding test set
            test_data = class_df.drop(train_indices)
            X_test = test_data[covariates]
            y_test = test_data[target]

            # Store train and test sets
            train_test_data[f"{method}_{level}_set_{t}"] = (
                X_train,
                y_train,
                X_test,
                y_test,
            )

# Generate SRS sample subsets
for level in sample_levels:
    for t in range(1, 21):
        # Simple random sampling
        srs_sample = resample(class_df, n_samples=level, random_state=42 + t)
        X_train = srs_sample[covariates]
        y_train = srs_sample[target]
        train_indices = srs_sample.index

        # Generate the corresponding test set
        test_data = class_df.drop(train_indices)
        X_test = test_data[covariates]
        y_test = test_data[target]

        # Store train and test sets
        train_test_data[f"SRS_sampled_df_{level}_set_{t}"] = (
            X_train,
            y_train,
            X_test,
            y_test,
        )

print("All training and testing sets generated and saved.")

All training and testing sets generated and saved.


In [ ]:
# Load the saved DataFrame files
results_df_rf = pd.read_csv("rfresults_icluROCAUC.csv")
results_df_lgb = pd.read_csv("lgbresults_icluROCAUC.csv")

In [ ]:
# Random Forest comparison: SRS vs FSCS at 20000 sample size for multi-class problem
import matplotlib.pyplot as plt
import joblib
import fasttreeshap

# Create a new variable to store extracted sampling method and dataset level
results_with_methods = results_df_rf.copy()

results_with_methods["sampling_method"] = results_with_methods["dataset"].apply(
    lambda x: x.split("_")[0]
)
results_with_methods["dataset_level"] = results_with_methods["dataset"].apply(
    lambda x: x.split("_")[-3]
)

results_with_methods["set_number"] = results_with_methods["dataset"].apply(
    lambda x: x.split("_")[-1]
)

# Calculate the average of three metrics
results_with_methods["average_metric"] = results_with_methods[
    ["test_accuracy", "test_f1", "roc_auc"]
].mean(axis=1)

# Find the median-performing SRS result at 20000 level
sorted_results_srs = results_with_methods[
    (results_with_methods["sampling_method"] == "SRS")
    & (results_with_methods["dataset_level"] == "20000")
].sort_values(by="average_metric", ascending=False)

# Find the median-performing FSCS result at 20000 level
sorted_results_fscs = results_with_methods[
    (results_with_methods["sampling_method"] == "FSCS")
    & (results_with_methods["dataset_level"] == "20000")
].sort_values(by="average_metric", ascending=False)

# Get the median value
median_metric_srs = sorted_results_srs["average_metric"].median()
median_metric = sorted_results_fscs["average_metric"].median()

# Find the result closest to the median
sorted_results_srs["metric_diff"] = (
    sorted_results_srs["average_metric"] - median_metric
).abs()
middle_result_srs = sorted_results_srs.loc[sorted_results_srs["metric_diff"].idxmin()]

sorted_results_fscs["metric_diff"] = (
    sorted_results_fscs["average_metric"] - median_metric
).abs()
middle_result_fscs = sorted_results_fscs.loc[
    sorted_results_fscs["metric_diff"].idxmin()
]

# Get the set number
set_number_srs = middle_result_srs["set_number"]
set_number_fscs = middle_result_fscs["set_number"]

# Build file paths
file_path_srs = (
    f"codepart\\RFpkl\\SRS_sampled_df_20000_set_{set_number_srs}_shap_values_v2.pkl"
)
model_path_srs = f"codepart\\RFpkl\\SRS_sampled_df_20000_set_{set_number_srs}_model.pkl"

# Load pkl files
shap_values_srs_rf = joblib.load(file_path_srs)
clf_model_srs_rf = joblib.load(model_path_srs)

# Build file paths
file_path_fscs = (
    f"codepart\\RFpkl\\FSCS_sampled_data_20000_set_{set_number_fscs}_shap_values_v2.pkl"
)
model_path_fscs = (
    f"codepart\\RFpkl\\FSCS_sampled_data_20000_set_{set_number_fscs}_model.pkl"
)

# Load pkl files
shap_values_fscs_rf = joblib.load(file_path_fscs)
clf_model_fscs_rf = joblib.load(model_path_fscs)

# Build dataset keys
dataset_srs = f"SRS_sampled_df_20000_set_{set_number_srs}"
dataset_fscs = f"FSCS_sampled_data_20000_set_{set_number_srs}"

# Get the corresponding datasets
X_train_srs_rf, y_train_srs_rf, X_test_srs_rf, y_test_srs_rf = train_test_data[
    dataset_srs
]
X_train_fscs_rf, y_train_fscs_rf, X_test_fscs_rf, y_test_fscs_rf = train_test_data[
    dataset_fscs
]

y_pred_srs_rf = clf_model_srs_rf.predict(X_test_srs_rf)
y_pred_fscs_rf = clf_model_fscs_rf.predict(X_test_fscs_rf)

In [ ]:
# LightGBM comparison: SRS vs FSCS at 20000 sample size for multi-class problem
import joblib
import fasttreeshap
from sklearn.preprocessing import LabelEncoder

# Create a new variable to store extracted sampling method and dataset level
results_with_methods = results_df_lgb.copy()

results_with_methods["sampling_method"] = results_with_methods["dataset"].apply(
    lambda x: x.split("_")[0]
)
results_with_methods["dataset_level"] = results_with_methods["dataset"].apply(
    lambda x: x.split("_")[-3]
)

results_with_methods["set_number"] = results_with_methods["dataset"].apply(
    lambda x: x.split("_")[-1]
)

# Calculate the average of three metrics
results_with_methods["average_metric"] = results_with_methods[
    ["test_accuracy", "test_f1", "roc_auc"]
].mean(axis=1)

# Find the median-performing SRS result at 20000 level
sorted_results_srs = results_with_methods[
    (results_with_methods["sampling_method"] == "SRS")
    & (results_with_methods["dataset_level"] == "20000")
].sort_values(by="average_metric", ascending=False)

# Find the median-performing FSCS result at 20000 level
sorted_results_fscs = results_with_methods[
    (results_with_methods["sampling_method"] == "FSCS")
    & (results_with_methods["dataset_level"] == "20000")
].sort_values(by="average_metric", ascending=False)

# Get the median value
median_metric_srs = sorted_results_srs["average_metric"].median()
median_metric = sorted_results_fscs["average_metric"].median()

# Find the result closest to the median
sorted_results_srs["metric_diff"] = (
    sorted_results_srs["average_metric"] - median_metric
).abs()
middle_result_srs = sorted_results_srs.loc[sorted_results_srs["metric_diff"].idxmin()]

sorted_results_fscs["metric_diff"] = (
    sorted_results_fscs["average_metric"] - median_metric
).abs()
middle_result_fscs = sorted_results_fscs.loc[
    sorted_results_fscs["metric_diff"].idxmin()
]

# Get the set number
set_number_srs = middle_result_srs["set_number"]
set_number_fscs = middle_result_fscs["set_number"]

# Build model file paths
file_path_srs = f"codepart\\LGBpkl\\SRS_sampled_df_20000_set_{set_number_srs}_model.pkl"
clf_srs = joblib.load(file_path_srs)
file_path_fscs = (
    f"codepart\\LGBpkl\\FSCS_sampled_data_20000_set_{set_number_fscs}_model.pkl"
)
clf_fscs = joblib.load(file_path_fscs)

# Build dataset keys
dataset_srs = f"SRS_sampled_df_20000_set_{set_number_srs}"
dataset_fscs = f"FSCS_sampled_data_20000_set_{set_number_fscs}"

# Get the corresponding datasets
X_train_srs_lgb, y_train_srs_lgb, X_test_srs_lgb, y_test_srs_lgb = train_test_data[
    dataset_srs
]
X_train_fscs_lgb, y_train_fscs_lgb, X_test_fscs_lgb, y_test_fscs_lgb = train_test_data[
    dataset_fscs
]

# Calculate SHAP values
# It can be modified to read directly, but fasttreeshap is also fast.
shap_explainer_srs = fasttreeshap.TreeExplainer(
    clf_srs, algorithm="v0", n_jobs=-1, shortcut=True
)
shap_values_srs_lgb = shap_explainer_srs(X_train_srs_lgb).values

shap_explainer_fscs = fasttreeshap.TreeExplainer(
    clf_fscs, algorithm="v0", n_jobs=-1, shortcut=True
)
shap_values_fscs_lgb = shap_explainer_fscs(X_train_fscs_lgb).values

# Label encoding
le = LabelEncoder()
y_train_srs_lgb = le.fit_transform(y_train_srs_lgb)
y_train_srs_lgb = le.inverse_transform(y_train_srs_lgb)

# Predict test set labels for SRS
y_pred_srs_lgb_proba = clf_srs.predict(X_test_srs_lgb)
y_pred_srs_lgb = np.argmax(y_pred_srs_lgb_proba, axis=1)
y_pred_srs_lgb = le.inverse_transform(y_pred_srs_lgb)

# Predict test set labels for FSCS
y_pred_fscs_lgb_proba = clf_fscs.predict(X_test_fscs_lgb)
y_pred_fscs_lgb = np.argmax(y_pred_fscs_lgb_proba, axis=1)
y_pred_fscs_lgb = le.inverse_transform(y_pred_fscs_lgb)

In [ ]:
# Plot the distribution of SHAP feature importance for each class

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import joblib

# Calculate the mean absolute SHAP value for each feature in each class
shap_values_srs_rf_abs_mean = np.mean(np.abs(shap_values_srs_rf), axis=0)
shap_values_fscs_rf_abs_mean = np.mean(np.abs(shap_values_fscs_rf), axis=0)
shap_values_srs_lgb_abs_mean = np.mean(np.abs(shap_values_srs_lgb), axis=0)
shap_values_fscs_lgb_abs_mean = np.mean(np.abs(shap_values_fscs_lgb), axis=0)
shap_values_srs_rf_abs_mean = shap_values_srs_rf_abs_mean / np.sum(
    shap_values_srs_rf_abs_mean
)
shap_values_fscs_rf_abs_mean = shap_values_fscs_rf_abs_mean / np.sum(
    shap_values_fscs_rf_abs_mean
)
shap_values_srs_lgb_abs_mean = shap_values_srs_lgb_abs_mean / np.sum(
    shap_values_srs_lgb_abs_mean
)
shap_values_fscs_lgb_abs_mean = shap_values_fscs_lgb_abs_mean / np.sum(
    shap_values_fscs_lgb_abs_mean
)

# Class names
class_names = [
    "Aspen",
    "Cottonwood Willow",
    "Douglas Fir",
    "Krummholz",
    "Lodgepole Pine",
    "Ponderosa Pine",
    "Spruce Fir",
]

# Manually specify x-axis labels
feature_names = [
    "Elevation",
    "Aspect",
    "Slope",
    "Horizontal Distance\nto Hydrology",
    "Vertical Distance\nto Hydrology",
    "Horizontal Distance\nto Roadways",
    "Hillshade 9am",
    "Hillshade Noon",
    "Hillshade 3pm",
    "Horizontal Distance\nto Fire Points",
]

# Add sampling method names to feature names
feature_names_srs_rf = [f"RF+SRS-{feature}" for feature in feature_names]
feature_names_fscs_rf = [f"RF+FSCS-{feature}" for feature in feature_names]
feature_names_srs_lgb = [f"LGB+SRS-{feature}" for feature in feature_names]
feature_names_fscs_lgb = [f"LGB+FSCS-{feature}" for feature in feature_names]

# Interleave feature names
combined_feature_names = []
combined_shap_values_abs_mean = []

for srs_rf_feature, fscs_rf_feature, srs_lgb_feature, fscs_lgb_feature in zip(
    feature_names_srs_rf,
    feature_names_fscs_rf,
    feature_names_srs_lgb,
    feature_names_fscs_lgb,
):
    combined_feature_names.append(srs_rf_feature)
    combined_feature_names.append(fscs_rf_feature)
    combined_feature_names.append(srs_lgb_feature)
    combined_feature_names.append(fscs_lgb_feature)

for srs_rf_shap, fscs_rf_shap, srs_lgb_shap, fscs_lgb_shap in zip(
    shap_values_srs_rf_abs_mean,
    shap_values_fscs_rf_abs_mean,
    shap_values_srs_lgb_abs_mean,
    shap_values_fscs_lgb_abs_mean,
):
    combined_shap_values_abs_mean.append(srs_rf_shap)
    combined_shap_values_abs_mean.append(fscs_rf_shap)
    combined_shap_values_abs_mean.append(srs_lgb_shap)
    combined_shap_values_abs_mean.append(fscs_lgb_shap)

combined_shap_values_abs_mean = np.array(combined_shap_values_abs_mean)

# Plot stacked bar chart
num_classes = shap_values_srs_rf.shape[2]
num_features = len(feature_names)
x = np.arange(num_features)
width = 0.35  # Adjust width to fit four bar plots

fig, ax = plt.subplots(figsize=(10, 8))
bottom_srs_rf = np.zeros(num_features)
bottom_fscs_rf = np.zeros(num_features)
bottom_srs_lgb = np.zeros(num_features)
bottom_fscs_lgb = np.zeros(num_features)

# Plot SHAP values for SRS and FSCS
for i in range(num_classes):
    bar_srs_rf = ax.bar(
        x - 3 * width / 4,
        shap_values_srs_rf_abs_mean[:, i],
        width / 2,
        label=f"{class_names[i]}",
        bottom=bottom_srs_rf,
    )
    bar_fscs_rf = ax.bar(
        x - width / 4,
        shap_values_fscs_rf_abs_mean[:, i],
        width / 2,
        bottom=bottom_fscs_rf,
        color=[bar.get_facecolor() for bar in bar_srs_rf],
    )
    bar_srs_lgb = ax.bar(
        x + width / 4,
        shap_values_srs_lgb_abs_mean[:, i],
        width / 2,
        bottom=bottom_srs_lgb,
        color=[bar.get_facecolor() for bar in bar_srs_rf],
    )
    bar_fscs_lgb = ax.bar(
        x + 3 * width / 4,
        shap_values_fscs_lgb_abs_mean[:, i],
        width / 2,
        bottom=bottom_fscs_lgb,
        color=[bar.get_facecolor() for bar in bar_srs_rf],
    )
    bottom_srs_rf += shap_values_srs_rf_abs_mean[:, i]
    bottom_fscs_rf += shap_values_fscs_rf_abs_mean[:, i]
    bottom_srs_lgb += shap_values_srs_lgb_abs_mean[:, i]
    bottom_fscs_lgb += shap_values_fscs_lgb_abs_mean[:, i]

# Add annotations
ax.text(
    x[0] - 3 * width / 4,
    bottom_srs_rf[0],
    "RF\nSRS",  # Line break annotation
    ha="center",
    va="bottom",
    fontsize=9,
)
ax.text(
    x[0] - width / 4,
    bottom_fscs_rf[0] - 0.005,
    "RF\nFSCS",  # Line break annotation
    ha="center",
    va="bottom",
    fontsize=9,
)
ax.text(
    x[0] + width / 4,
    bottom_srs_lgb[0] + 0.01,
    "LGB\nSRS",  # Line break annotation
    ha="center",
    va="bottom",
    fontsize=9,
)
ax.text(
    x[0] + 3 * width / 4,
    bottom_fscs_lgb[0] - 0.01,
    "LGB\nFSCS",  # Line break annotation
    ha="center",
    va="bottom",
    fontsize=9,
)

# Beautify the chart
ax.set_xlabel("Features", fontsize=14)
ax.set_ylabel("Normalization Mean |SHAP Value|", fontsize=14)
ax.set_title("SHAP Feature Importance for RF/LGB + SRS/FSCS", fontsize=18)
ax.set_xticks(x)
ax.set_xticklabels(feature_names, rotation=45, ha="right", fontsize=14)
ax.legend(title="Classes", fontsize=14, title_fontsize=16)
ax.grid(True, linestyle="--", alpha=0.7)
plt.grid(True, which="major", axis="y", linestyle="-", linewidth=0.6)
plt.grid(True, which="minor", axis="y", linestyle="--", linewidth=0.25)
plt.gca().yaxis.set_major_locator(plt.MultipleLocator(0.05))
plt.gca().yaxis.set_minor_locator(plt.MultipleLocator(0.01))

plt.tight_layout()
plt.savefig("shapclass.jpg", format="jpg", dpi=800, bbox_inches="tight")
plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import joblib

# Calculate the mean absolute SHAP value for each feature in each class
shap_values_srs_rf_abs_mean = np.mean(np.abs(shap_values_srs_rf), axis=0)
shap_values_fscs_rf_abs_mean = np.mean(np.abs(shap_values_fscs_rf), axis=0)
shap_values_srs_lgb_abs_mean = np.mean(np.abs(shap_values_srs_lgb), axis=0)
shap_values_fscs_lgb_abs_mean = np.mean(np.abs(shap_values_fscs_lgb), axis=0)

# Normalize
shap_values_srs_rf_abs_mean /= shap_values_srs_rf_abs_mean.sum()
shap_values_fscs_rf_abs_mean /= shap_values_fscs_rf_abs_mean.sum()
shap_values_srs_lgb_abs_mean /= shap_values_srs_lgb_abs_mean.sum()
shap_values_fscs_lgb_abs_mean /= shap_values_fscs_lgb_abs_mean.sum()

# Class and feature names
class_names = [
    "Aspen",
    "Cottonwood Willow",
    "Douglas Fir",
    "Krummholz",
    "Lodgepole Pine",
    "Ponderosa Pine",
    "Spruce Fir",
]
feature_names = [
    "Elevation",
    "Aspect",
    "Slope",
    "Horizontal Distance to Hydrology",
    "Vertical Distance to Hydrology",
    "Horizontal Distance to Roadways",
    "Hillshade 9am",
    "Hillshade Noon",
    "Hillshade 3pm",
    "Horizontal Distance to Fire Points",
]

# X axis positions and bar width
num_features = len(feature_names)
x = np.arange(num_features)
width = 0.5

# Create 2x2 subplots, share Y axis
fig, axes = plt.subplots(2, 2, figsize=(12, 12), sharey=True)
axes = axes.flatten()
methods = [
    (shap_values_srs_rf_abs_mean, "(a) RF + SRS"),
    (shap_values_fscs_rf_abs_mean, "(b) RF + FSCS"),
    (shap_values_srs_lgb_abs_mean, "(c) LGB + SRS"),
    (shap_values_fscs_lgb_abs_mean, "(d) LGB + FSCS"),
]

for idx, (shap_vals, title) in enumerate(methods):
    ax = axes[idx]
    bottom = np.zeros(num_features)
    for i, cls in enumerate(class_names):
        values = shap_vals[:, i]
        ax.bar(
            x,
            values,
            width,
            bottom=bottom,
            label=cls,
        )
        bottom += values
    ax.set_title(title, fontsize=15)
    ax.grid(True, which="major", axis="y", linestyle="-", linewidth=0.6)
    ax.grid(True, which="minor", axis="y", linestyle="--", linewidth=0.25)
    ax.yaxis.set_major_locator(plt.MultipleLocator(0.05))
    ax.yaxis.set_minor_locator(plt.MultipleLocator(0.01))
    ax.tick_params(axis="y", labelsize=13)

    # Only show X axis ticks and labels on the second row
    if idx // 2 == 1:
        ax.set_xticks(x)
        ax.set_xticklabels(feature_names, rotation=45, ha="right", fontsize=13)
        ax.set_xlabel("Features", fontsize=13)
    else:
        ax.set_xticks(x)
        ax.set_xticklabels([])

    # Only show Y axis label on the first column
    if idx % 2 == 0:
        ax.set_ylabel("Normalized Mean |SHAP Value|", fontsize=13)

# Overall beautification
fig.suptitle(
    "SHAP Feature Importance by Model + Sampling Method and Class", fontsize=19
)
# Shared legend
handles, labels = axes[0].get_legend_handles_labels()
fig.legend(
    handles,
    labels,
    title="Classes",
    bbox_to_anchor=(0.98, 0.92),
    title_fontsize=14,
    fontsize=12,
)

plt.tight_layout()
fig.subplots_adjust(
    wspace=0.02, hspace=0.1
)  # Adjust horizontal and vertical spacing between subplots
plt.savefig("shap_class_fourpanels.png", dpi=1000, bbox_inches="tight")

In [ ]:
# Generate SHAP beeswarm plots
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import joblib
import shap

# Calculate the mean SHAP value for each feature in each class
shap_values_srs_rf_mean = np.mean(shap_values_srs_rf, axis=0)
shap_values_fscs_rf_mean = np.mean(shap_values_fscs_rf, axis=0)
shap_values_srs_lgb_mean = np.mean(shap_values_srs_lgb, axis=0)
shap_values_fscs_lgb_mean = np.mean(shap_values_fscs_lgb, axis=0)

# Class names
class_names = [
    "Aspen",
    "Cottonwood Willow",
    "Douglas Fir",
    "Krummholz",
    "Lodgepole Pine",
    "Ponderosa Pine",
    "Spruce Fir",
]

# Manually specify x-axis labels
feature_names = [
    "Elevation",
    "Aspect",
    "Slope",
    "Horizontal Distance\nto Hydrology",
    "Vertical Distance\nto Hydrology",
    "Horizontal Distance\nto Roadways",
    "Hillshade 9 am",
    "Hillshade Noon",
    "Hillshade 3 pm",
    "Horizontal Distance\nto Fire Points",
]

# Set global font size
plt.rcParams.update({"font.size": 16})


# Generate beeswarm plots for each class and save as image files
def plot_beeswarm(shap_values, X, class_name, method_name, filename):
    shap.summary_plot(shap_values, X, feature_names=feature_names, show=False)
    plt.title(f"SHAP Values for {class_name} ({method_name})", fontsize=20)
    plt.tight_layout()
    plt.xlabel("SHAP Value (Impact on Model Output)", fontsize=17)
    plt.savefig(filename, dpi=800)
    plt.close()


# Generate beeswarm plots for SRS and FSCS and save as image files
for i, class_name in enumerate(class_names):
    plot_beeswarm(
        shap_values_srs_rf[:, :, i],
        X_train_srs_rf,
        class_name,
        "RF+SRS",
        f"shap_srs_rf_{i}.png",
    )
    plot_beeswarm(
        shap_values_fscs_rf[:, :, i],
        X_train_fscs_rf,
        class_name,
        "RF+FSCS",
        f"shap_fscs_rf_{i}.png",
    )
    plot_beeswarm(
        shap_values_srs_lgb[:, :, i],
        X_train_srs_lgb,
        class_name,
        "LGB+SRS",
        f"shap_srs_lgb_{i}.png",
    )
    plot_beeswarm(
        shap_values_fscs_lgb[:, :, i],
        X_train_fscs_lgb,
        class_name,
        "LGB+FSCS",
        f"shap_fscs_lgb_{i}.png",
    )

In [ ]:
# Create and save the combined image for RF+SRS
fig, axes = plt.subplots(nrows=4, ncols=2, figsize=(10, 16))
for i, class_name in enumerate(class_names):
    row, col = divmod(i, 2)
    axes[row, col].imshow(plt.imread(f"shap_srs_rf_{i}.png"))
    axes[row, col].axis("off")
# Hide the last empty axis if the number of classes is odd
if len(class_names) % 2 != 0:
    axes[-1, -1].axis("off")
plt.tight_layout(pad=0)
plt.subplots_adjust(wspace=0, hspace=0)
plt.savefig("combined_beeswarm_rf_srs.png", dpi=800)
plt.show()

import matplotlib.pyplot as plt

# Create and save the combined image for RF+FSCS
fig, axes = plt.subplots(nrows=4, ncols=2, figsize=(10, 16))
for i, class_name in enumerate(class_names):
    row, col = divmod(i, 2)
    axes[row, col].imshow(plt.imread(f"shap_fscs_rf_{i}.png"))
    axes[row, col].axis("off")
# Hide the last empty axis if the number of classes is odd
if len(class_names) % 2 != 0:
    axes[-1, -1].axis("off")
plt.tight_layout(pad=0)
plt.subplots_adjust(wspace=0, hspace=0)
plt.savefig("combined_beeswarm_rf_fscs.png", dpi=800)
plt.show()

# Create and save the combined image for LGB+SRS
fig, axes = plt.subplots(nrows=4, ncols=2, figsize=(10, 16))
for i, class_name in enumerate(class_names):
    row, col = divmod(i, 2)
    axes[row, col].imshow(plt.imread(f"shap_srs_lgb_{i}.png"))
    axes[row, col].axis("off")
# Hide the last empty axis if the number of classes is odd
if len(class_names) % 2 != 0:
    axes[-1, -1].axis("off")
plt.tight_layout(pad=0)
plt.subplots_adjust(wspace=0, hspace=0)
plt.savefig("combined_beeswarm_lgb_srs.png", dpi=800)
plt.show()

# Create and save the combined image for LGB+FSCS
fig, axes = plt.subplots(nrows=4, ncols=2, figsize=(10, 16))
for i, class_name in enumerate(class_names):
    row, col = divmod(i, 2)
    axes[row, col].imshow(plt.imread(f"shap_fscs_lgb_{i}.png"))
    axes[row, col].axis("off")
# Hide the last empty axis if the number of classes is odd
if len(class_names) % 2 != 0:
    axes[-1, -1].axis("off")
plt.tight_layout(pad=0)
plt.subplots_adjust(wspace=0, hspace=0)
plt.savefig("combined_beeswarm_lgb_fscs.png", dpi=800)

In [ ]:
# Target variable distribution
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Create a DataFrame for all data
df_overall = pd.DataFrame({"Class": class_df[target], "Dataset": "Overall"})
df_srs_rf = pd.DataFrame({"Class": y_train_srs_rf, "Dataset": "RF+SRS Train"})
df_fscs_rf = pd.DataFrame({"Class": y_train_fscs_rf, "Dataset": "RF+FSCS Train"})
df_srs_lgb = pd.DataFrame({"Class": y_train_srs_lgb, "Dataset": "LGB+SRS Train"})
df_fscs_lgb = pd.DataFrame({"Class": y_train_fscs_lgb, "Dataset": "LGB+FSCS Train"})

# Combine all data
df_combined = pd.concat([df_overall, df_srs_rf, df_fscs_rf, df_srs_lgb, df_fscs_lgb])

# Convert the Class column to an ordered categorical type
class_names = [
    "Aspen",
    "Cottonwood_Willow",
    "Douglas_fir",
    "Krummholz",
    "Lodgepole_Pine",
    "Ponderosa_Pine",
    "Spruce_Fir",
]

class_labels = [
    "Aspen",
    "Cottonwood Willow",
    "Douglas Fir",
    "Krummholz",
    "Lodgepole Pine",
    "Ponderosa Pine",
    "Spruce Fir",
]
df_combined["Class"] = pd.Categorical(
    df_combined["Class"], categories=class_names, ordered=True
)

# Set plot style
sns.set(style="whitegrid")

# Create a histplot
plt.figure(figsize=(12, 8))
sns.histplot(
    data=df_combined,
    x="Class",
    hue="Dataset",
    stat="density",
    common_norm=False,
    element="bars",
    multiple="dodge",
    shrink=0.8,
)

# Set x-axis category order
plt.xticks(ticks=range(len(class_names)), labels=class_labels)
plt.yticks(fontsize=12)

# Add title and labels
plt.title("Class Distribution by Sampling Method (Density)", fontsize=15)
plt.xlabel("Classes", fontsize=14)
plt.ylabel("Density", fontsize=14)
plt.savefig("Class_Distribution_compare.png", dpi=800)

# Show the plot
plt.show()

In [ ]:
# Covariate feature distributions
# Please note that this image was saved manually. Using plt.savefig caused the figure to become distorted.

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Define covariate features and target variable
features = [
    "elevation",
    "aspect",
    "slope",
    "horizontal_distance_to_hydrology",
    "Vertical_Distance_To_Hydrology",
    "Horizontal_Distance_To_Roadways",
    "Hillshade_9am",
    "Hillshade_Noon",
    "Hillshade_3pm",
    "Horizontal_Distance_To_Fire_Points",
]

feature_names = [
    "Elevation",
    "Aspect",
    "Slope",
    "Horizontal Distance\nto Hydrology",
    "Vertical Distance\nto Hydrology",
    "Horizontal Distance\nto Roadways",
    "Hillshade 9 am",
    "Hillshade Noon",
    "Hillshade 3 pm",
    "Horizontal Distance\nto Fire Points",
]

# Create a combined DataFrame containing all datasets
df_combined = pd.concat(
    [
        class_df.assign(Dataset="Overall"),
        X_train_srs_rf.assign(Dataset="RF+SRS Train"),
        X_train_fscs_rf.assign(Dataset="RF+FSCS Train"),
        X_train_srs_lgb.assign(Dataset="LGB+SRS Train"),
        X_train_fscs_lgb.assign(Dataset="LGB+FSCS Train"),
    ]
)

# Define different linestyles and linewidths for each dataset
linestyles = {
    "Overall": "-",
    "RF+SRS Train": "--",
    "RF+FSCS Train": "-.",
    "LGB+SRS Train": "--",
    "LGB+FSCS Train": "-.",
}
linewidths = {
    "Overall": 3.5,
    "RF+SRS Train": 2.5,
    "RF+FSCS Train": 2.5,
    "LGB+SRS Train": 2.5,
    "LGB+FSCS Train": 2.5,
}

# Define colors for each dataset
colors = {
    "Overall": "black",
    "RF+SRS Train": "red",
    "RF+FSCS Train": "blue",
    "LGB+SRS Train": "darkred",
    "LGB+FSCS Train": "darkblue",
}

# Create a 4x3 subplot layout, one subplot for each feature
fig, axes = plt.subplots(4, 3, figsize=(25, 30))
sns.set(style="whitegrid")

for i, (feature, feature_name) in enumerate(zip(features, feature_names)):
    ax = axes[i // 3, i % 3]
    df_combined_data = df_combined[[feature, "Dataset"]]
    for j, (dataset, linestyle) in enumerate(
        zip(
            [
                "Overall",
                "RF+SRS Train",
                "RF+FSCS Train",
                "LGB+SRS Train",
                "LGB+FSCS Train",
            ],
            linestyles,
        )
    ):
        g = sns.kdeplot(
            data=df_combined_data[df_combined_data["Dataset"] == dataset],
            x=feature,
            label=dataset,
            ax=ax,
            linestyle=linestyles[dataset],
            linewidth=linewidths[dataset],
            color=colors[dataset],  # Specify color
        )
    ax.set_title(feature_name, fontsize=22)
    ax.set_xlabel("")
    ax.set_ylabel("Density", fontsize=20)
    ax.set_yscale("log")  # Set y-axis to log scale
    ax.grid(True)
    ax.tick_params(axis="both", which="major", labelsize=20)
    if i % 3 == 0:
        ax.set_ylabel("Density", fontsize=22)
    else:
        ax.set_ylabel("")
# Remove the last two empty subplots
axes[3, 1].axis("off")
axes[3, 2].axis("off")
# Place the legend outside the first subplot, to the upper left
handles, labels = ax.get_legend_handles_labels()
fig.legend(
    handles,
    labels,
    loc="upper center",
    bbox_to_anchor=(0.43, 1.031),
    fontsize=21,
    title="Dataset",
    title_fontsize=21,
    ncol=5,
)

# Adjust layout
plt.tight_layout(rect=[0, 0, 0.85, 1])

# plt.savefig("Class_Feature_Distribution_compare.png", dpi=800)
# The code is commented out due to savefig deformation.

# Show the plot
plt.show()

The following is the Ksat drawing.

In [ ]:
# Data loading
import pandas as pd
import numpy as np
import os
from sklearn.utils import resample

# Read the original dataset
class_df = pd.read_csv(
    "sampleddata/USKSAT_OpenRefined_cleaned.csv",
    index_col=0,  # Use the first column as index
)

# Convert the target variable to natural logarithm
class_df["Ksat_cmhr"] = np.log(class_df["Ksat_cmhr"])

# Define sample sizes and sampling methods
sample_levels = [1000, 5000, 10000]

sampling_methods = [
    "BalancedSampling_sampled_data",
    "clhs_sampled_data",
    "FCMtp2sampled_data",
    "FSCS_sampled_data",
]

# Define covariates and target variable
covariates = [
    "Db",
    "Clay",
    "VFS",
    "MS",
    "OC",
    "Silt",
    "COS",
    "FS",
    "Depth.cm_Top",
    "VCOS",
]
covariates_FSCS = [
    "Db",
    "Clay",
    "VFS",
    "MS",
    "OC",
    "Silt",
    "COS",
    "FS",
    "Depth_cm_Top",
    "VCOS",
]
target = "Ksat_cmhr"

# Store train and test sets
train_test_data = {}

# Loop through each sample size and sampling method
for level in sample_levels:
    for t in range(1, 21):
        for method in sampling_methods:
            sample_file = (
                f"sampleddata/combined_samples_Ksat/{method}_{level}_set_{t}.csv"
            )

            if not os.path.exists(sample_file):
                print(f"File {sample_file} does not exist. Skipping...")
                continue

            sample_data = pd.read_csv(
                sample_file, index_col=0
            )  # Use the first column as index
            sample_data[target] = np.log(
                sample_data[target]
            )  # Convert the target variable to natural logarithm
            if method == "FSCS_sampled_data":
                X_train = sample_data[covariates_FSCS]
                X_train = X_train.rename(columns={"Depth_cm_Top": "DT"})
            else:
                X_train = sample_data[covariates]
                X_train = X_train.rename(columns={"Depth.cm_Top": "DT"})

            y_train = sample_data[target]
            y_train.name = "Ks"
            train_indices = sample_data.index

            # Generate the corresponding test set
            test_data = class_df.drop(train_indices)
            X_test = test_data[covariates]
            X_test = X_test.rename(columns={"Depth.cm_Top": "DT"})
            y_test = test_data[target]
            y_test.name = "Ks"

            # Store train and test sets
            train_test_data[f"{method}_{level}_set_{t}"] = (
                X_train,
                y_train,
                X_test,
                y_test,
            )

# Generate SRS sample subsets
for level in sample_levels:
    for t in range(1, 21):
        # Simple random sampling
        srs_sample = resample(class_df, n_samples=level, random_state=42 + t)
        X_train = srs_sample[covariates]
        y_train = srs_sample[target]
        train_indices = srs_sample.index

        # Generate the corresponding test set
        test_data = class_df.drop(train_indices)
        X_test = test_data[covariates]
        y_test = test_data[target]

        X_train = X_train.rename(columns={"Depth.cm_Top": "DT"})
        X_test = X_test.rename(columns={"Depth.cm_Top": "DT"})
        y_train.name = "Ks"
        y_test.name = "Ks"

        # Store train and test sets
        train_test_data[f"SRS_sampled_df_{level}_set_{t}"] = (
            X_train,
            y_train,
            X_test,
            y_test,
        )

print("All training and testing sets generated and saved.")

All training and testing sets generated and saved.


In [ ]:
# Load the saved DataFrame files
results_df_rfksat = pd.read_csv("rfKsatresults_all.csv")
results_df_lgbksat = pd.read_csv("lgbKsatresults_all.csv")

In [ ]:
# Random Forest comparison: SRS vs FSCS at 10000 sample level for regression (continuous variable)
import matplotlib.pyplot as plt
import joblib
import fasttreeshap

# Create a new variable to store extracted sampling method and dataset level
results_with_methods = results_df_rfksat.copy()

results_with_methods["sampling_method"] = results_with_methods["dataset"].apply(
    lambda x: x.split("_")[0]
)
results_with_methods["dataset_level"] = results_with_methods["dataset"].apply(
    lambda x: x.split("_")[-3]
)

results_with_methods["set_number"] = results_with_methods["dataset"].apply(
    lambda x: x.split("_")[-1]
)

# Calculate the average of two metrics (R2 and adjusted RMSLE)
results_with_methods["adjusted_rmsle"] = 1 - results_with_methods["rmsle"]
results_with_methods["average_metric"] = results_with_methods[
    ["r2", "adjusted_rmsle"]
].mean(axis=1)

# Find the median-performing result for SRS at 10000 level
sorted_results_srs = results_with_methods[
    (results_with_methods["sampling_method"] == "SRS")
    & (results_with_methods["dataset_level"] == "10000")
].sort_values(by="average_metric", ascending=False)

# Find the median-performing result for FSCS at 10000 level
sorted_results_fscs = results_with_methods[
    (results_with_methods["sampling_method"] == "FSCS")
    & (results_with_methods["dataset_level"] == "10000")
].sort_values(by="average_metric", ascending=False)

# Get the median value for each method
median_metric_srs = sorted_results_srs["average_metric"].median()
median_metric = sorted_results_fscs["average_metric"].median()

# Find the result closest to the median for each method
sorted_results_srs["metric_diff"] = (
    sorted_results_srs["average_metric"] - median_metric
).abs()
middle_result_srs = sorted_results_srs.loc[sorted_results_srs["metric_diff"].idxmin()]

sorted_results_fscs["metric_diff"] = (
    sorted_results_fscs["average_metric"] - median_metric
).abs()
middle_result_fscs = sorted_results_fscs.loc[
    sorted_results_fscs["metric_diff"].idxmin()
]

# Get the set number for each selected result
set_number_srs = middle_result_srs["set_number"]
set_number_fscs = middle_result_fscs["set_number"]

# Build file path for SRS SHAP values
file_path_srs = (
    f"codepart\\RFKsatpkl\\SRS_sampled_df_10000_set_{set_number_srs}_shap_values_v2.pkl"
)

# Load SRS SHAP values from pkl file
shap_values_srs_rf = joblib.load(file_path_srs)

# Build file path for FSCS SHAP values
file_path_fscs = f"codepart\\RFKsatpkl\\FSCS_sampled_data_10000_set_{set_number_fscs}_shap_values_v2.pkl"

# Load FSCS SHAP values from pkl file
shap_values_fscs_rf = joblib.load(file_path_fscs)

# Build file path for SRS SHAP interaction values
file_path_srs_inter = f"codepart\\RFKsatpkl\\SRS_sampled_df_10000_set_{set_number_srs}_shap_interaction_values_v1.pkl"

# Load SRS SHAP interaction values from pkl file
shap_values_srs_rf_inter = joblib.load(file_path_srs_inter)

# Build file path for FSCS SHAP interaction values
file_path_fscs_inter = f"codepart\\RFKsatpkl\\FSCS_sampled_data_10000_set_{set_number_fscs}_shap_interaction_values_v1.pkl"

# Load FSCS SHAP interaction values from pkl file
shap_values_fscs_rf_inter = joblib.load(file_path_fscs_inter)

# Build dataset keys
dataset_srs = f"SRS_sampled_df_10000_set_{set_number_srs}"
dataset_fscs = f"FSCS_sampled_data_10000_set_{set_number_srs}"

# Retrieve corresponding datasets
X_train_srs_rf, y_train_srs_rf, X_test_srs_rf, y_test_srs_rf = train_test_data[
    dataset_srs
]
X_train_fscs_rf, y_train_fscs_rf, X_test_fscs_rf, y_test_fscs_rf = train_test_data[
    dataset_fscs
]

In [ ]:
# Plot RF beeswarm plots

import shap

# Manually specify x-axis labels
feature_names = [
    "Db",
    "Clay",
    "VFS",
    "MS",
    "OC",
    "Silt",
    "COS",
    "FS",
    "Depth_cm_Top",
    "VCOS",
]

shap.summary_plot(
    shap_values_srs_rf, X_train_srs_rf, feature_names=feature_names, show=False
)
plt.title(f"SHAP Values for RF + SRS")
plt.tight_layout()
plt.savefig("beeswarm_rfksat_srs.png", dpi=800)
plt.show()

shap.summary_plot(
    shap_values_fscs_rf, X_train_fscs_rf, feature_names=feature_names, show=False
)
plt.title(f"SHAP Values for RF + FSCS")
plt.tight_layout()
plt.savefig("beeswarm_rfksat_fscs.png", dpi=800)
plt.show()

In [ ]:
# LightGBM comparison: SRS vs FSCS at 10000 sample level for regression (continuous variable)
import joblib
import fasttreeshap
from sklearn.preprocessing import LabelEncoder

# Create a new variable to store extracted sampling method and dataset level
results_with_methods = results_df_lgbksat.copy()

results_with_methods["sampling_method"] = results_with_methods["dataset"].apply(
    lambda x: x.split("_")[0]
)
results_with_methods["dataset_level"] = results_with_methods["dataset"].apply(
    lambda x: x.split("_")[-3]
)

results_with_methods["set_number"] = results_with_methods["dataset"].apply(
    lambda x: x.split("_")[-1]
)

# Calculate the average of two metrics (R2 and RMSLE)
results_with_methods["average_metric"] = results_with_methods[["r2", "rmsle"]].mean(
    axis=1
)

# Find the median-performing result for SRS at 10000 level
sorted_results_srs = results_with_methods[
    (results_with_methods["sampling_method"] == "SRS")
    & (results_with_methods["dataset_level"] == "10000")
].sort_values(by="average_metric", ascending=False)

# Find the median-performing result for FSCS at 10000 level
sorted_results_fscs = results_with_methods[
    (results_with_methods["sampling_method"] == "FSCS")
    & (results_with_methods["dataset_level"] == "10000")
].sort_values(by="average_metric", ascending=False)

# Get the median value for each method
median_metric_srs = sorted_results_srs["average_metric"].median()
median_metric = sorted_results_fscs["average_metric"].median()

# Find the result closest to the median for each method
sorted_results_srs["metric_diff"] = (
    sorted_results_srs["average_metric"] - median_metric
).abs()
middle_result_srs = sorted_results_srs.loc[sorted_results_srs["metric_diff"].idxmin()]

sorted_results_fscs["metric_diff"] = (
    sorted_results_fscs["average_metric"] - median_metric
).abs()
middle_result_fscs = sorted_results_fscs.loc[
    sorted_results_fscs["metric_diff"].idxmin()
]

# Get the set number for each selected result
set_number_srs = middle_result_srs["set_number"]
set_number_fscs = middle_result_fscs["set_number"]

# Build file paths
file_path_srs = (
    f"codepart\\LGBKsatpkl\\SRS_sampled_df_10000_set_{set_number_srs}_model.pkl"
)
clf_srs = joblib.load(file_path_srs)
file_path_fscs = (
    f"codepart\\LGBKsatpkl\\FSCS_sampled_data_10000_set_{set_number_fscs}_model.pkl"
)
clf_fscs = joblib.load(file_path_fscs)

# Build dataset keys
dataset_srs = f"SRS_sampled_df_10000_set_{set_number_srs}"
dataset_fscs = f"FSCS_sampled_data_10000_set_{set_number_fscs}"

# Retrieve corresponding datasets
(
    X_train_srs_lgb,
    y_train_srs_lgb,
    X_test_srs_lgb,
    y_test_srs_lgb,
) = train_test_data[dataset_srs]

X_train_fscs_lgb, y_train_fscs_lgb, X_test_fscs_lgb, y_test_fscs_lgb = train_test_data[
    dataset_fscs
]

# Calculate SHAP values and SHAP interaction values for SRS
shap_explainer_srs = fasttreeshap.TreeExplainer(
    clf_srs, algorithm="v0", n_jobs=-1, shortcut=True
)
shap_values_srs_lgb = shap_explainer_srs(X_train_srs_lgb).values
shap_values_srs_lgb_inter = shap_explainer_srs(
    X_train_srs_lgb, interactions=True
).values

# Calculate SHAP values and SHAP interaction values for FSCS
shap_explainer_srs = fasttreeshap.TreeExplainer(
    clf_srs, algorithm="v0", n_jobs=-1, shortcut=True
)
shap_values_srs_lgb = shap_explainer_srs(X_train_srs_lgb).values
shap_values_srs_lgb_inter = shap_explainer_srs(
    X_train_srs_lgb, interactions=True
).values

shap_explainer_fscs = fasttreeshap.TreeExplainer(
    clf_fscs, algorithm="v0", n_jobs=-1, shortcut=True
)
shap_values_fscs_lgb = shap_explainer_fscs(X_train_fscs_lgb).values
shap_values_fscs_lgb_inter = shap_explainer_fscs(
    X_train_fscs_lgb, interactions=True
).values

In [ ]:
# Plot LightGBM beeswarm plots

import shap

# Manually specify x-axis labels
feature_names = [
    "Db",
    "Clay",
    "VFS",
    "MS",
    "OC",
    "Silt",
    "COS",
    "FS",
    "DT",
    "VCOS",
]

shap.summary_plot(
    shap_values_srs_rf, X_train_srs_rf, feature_names=feature_names, show=False
)
plt.title(f"SHAP Values for RF + SRS", fontsize=20)
plt.xlabel("SHAP Value (Impact on Model Output)", fontsize=18)
plt.tight_layout()
plt.savefig("beeswarm_rfksat_srs.png", dpi=800)
plt.close()

shap.summary_plot(
    shap_values_fscs_rf, X_train_fscs_rf, feature_names=feature_names, show=False
)
plt.title(f"SHAP Values for RF + FSCS", fontsize=20)
plt.xlabel("SHAP Value (Impact on Model Output)", fontsize=18)
plt.tight_layout()
plt.savefig("beeswarm_rfksat_fscs.png", dpi=800)
plt.close()

shap.summary_plot(
    shap_values_srs_lgb, X_train_srs_lgb, feature_names=feature_names, show=False
)
plt.title(f"SHAP Values for LGB + SRS", fontsize=20)
plt.xlabel("SHAP Value (Impact on Model Output)", fontsize=18)
plt.tight_layout()
plt.savefig("beeswarm_lgbksat_srs.png", dpi=800)
plt.close()

shap.summary_plot(
    shap_values_fscs_lgb, X_train_fscs_lgb, feature_names=feature_names, show=False
)
plt.title(f"SHAP Values for LGB + FSCS", fontsize=20)
plt.xlabel("SHAP Value (Impact on Model Output)", fontsize=18)
plt.tight_layout()
plt.savefig("beeswarm_lgbksat_fscs.png", dpi=800)
plt.close()

In [ ]:
# Create a 2x2 layout, each subplot for one feature importance plot
fig, axes = plt.subplots(2, 2, figsize=(20, 15))


def plot_feature_importance(shap_values, X, feature_names, ax, title):
    shap_values_mean = np.abs(shap_values).mean(axis=0)
    sorted_idx = np.argsort(shap_values_mean)
    sorted_feature_names = np.array(feature_names)[sorted_idx]
    sorted_shap_values_mean = shap_values_mean[sorted_idx]
    ax.barh(sorted_feature_names, sorted_shap_values_mean)
    ax.set_title(title, fontsize=22)
    ax.tick_params(axis="both", which="major", labelsize=16)
    ax.set_xlabel("Mean |SHAP Value|", fontsize=18)


# Plot feature importance for RF + SRS
plot_feature_importance(
    shap_values_srs_rf,
    X_train_srs_rf,
    feature_names,
    axes[0, 0],
    "Feature Importance for RF + SRS",
)

# Plot feature importance for RF + FSCS
plot_feature_importance(
    shap_values_fscs_rf,
    X_train_fscs_rf,
    feature_names,
    axes[0, 1],
    "Feature Importance for RF + FSCS",
)

# Plot feature importance for LGB + SRS
plot_feature_importance(
    shap_values_srs_lgb,
    X_train_srs_lgb,
    feature_names,
    axes[1, 0],
    "Feature Importance for LGB + SRS",
)

# Plot feature importance for LGB + FSCS
plot_feature_importance(
    shap_values_fscs_lgb,
    X_train_fscs_lgb,
    feature_names,
    axes[1, 1],
    "Feature Importance for LGB + FSCS",
)

# Adjust layout
plt.tight_layout()

# Save the figure
plt.savefig("feature_importance_all_methods.png", dpi=800)

# Show the figure
plt.show()

In [ ]:
import seaborn as sns
from scipy.stats import anderson_ksamp
from scipy.stats import cramervonmises_2samp

variable_ori = "Ksat_cmhr"
variable_new = "Ks"

# Create a DataFrame containing all data
df_overall = pd.DataFrame({variable_new: class_df[variable_ori], "Dataset": "Overall"})
df_srs_rf = pd.DataFrame({variable_new: y_train_srs_rf, "Dataset": "RF+SRS Train"})
df_fscs_rf = pd.DataFrame({variable_new: y_train_fscs_rf, "Dataset": "RF+FSCS Train"})
df_srs_lgb = pd.DataFrame({variable_new: y_train_srs_lgb, "Dataset": "LGB+SRS Train"})
df_fscs_lgb = pd.DataFrame(
    {variable_new: y_train_fscs_lgb, "Dataset": "LGB+FSCS Train"}
)

# Concatenate all data
df_combined = pd.concat([df_overall, df_srs_rf, df_fscs_rf, df_srs_lgb, df_fscs_lgb])

# Set plot style
sns.set(style="whitegrid")

# Create a 5-row, 1-column layout
fig, axes = plt.subplots(5, 1, figsize=(10, 8), sharex=True)

# Plot the density and histogram for each group
for i, (dataset, ax) in enumerate(zip(df_combined["Dataset"].unique(), axes)):
    subset = df_combined[df_combined["Dataset"] == dataset]
    sns.histplot(
        data=subset,
        x=variable_new,
        stat="density",
        fill=True,
        bins=50,  # Set the number of histogram bins
        ax=ax,
    )
    sns.kdeplot(
        data=subset,
        x=variable_new,
        label=dataset,
        bw_adjust=0.5,
        fill=True,
        log_scale=(False, True),
        ax=ax,
    )
    ax.set_title(f"{dataset} Distribution")
    if dataset == "RF+FSCS Train":
        ax.set_ylabel("Density (Log Scale)")
    else:
        ax.set_ylabel("")

# Set x-axis label
axes[-1].set_xlabel("ln(Ks)")
# Set x-axis tick interval to 1
axes[-1].xaxis.set_major_locator(plt.MultipleLocator(1))

# Set the same y-axis range for all subplots
y_min, y_max = axes[0].get_ylim()
for ax in axes:
    ax.set_ylim(y_min, y_max)

# Adjust layout
plt.tight_layout()
plt.savefig("Ksat_Distribution_compare.png", dpi=800)
# Show the plot
plt.show()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Define covariates and target variable
features = [
    "Db",
    "Clay",
    "VFS",
    "MS",
    "OC",
    "Silt",
    "COS",
    "FS",
    "DT",
    "VCOS",
]

# Manually specify x-axis labels and units
feature_names = [
    "Db (g/cm$^3$)",
    "Clay (%)",
    "VFS (%)",
    "MS (%)",
    "OC (%)",
    "Silt (%)",
    "COS (%)",
    "FS (%)",
    "DT (cm)",
    "VCOS (%)",
]

class_df.rename(columns={"Depth.cm_Top": "DT"}, inplace=True)
# Create a DataFrame containing all data
df_combined = pd.concat(
    [
        class_df.assign(Dataset="Overall"),
        X_train_srs_rf.assign(Dataset="RF+SRS Train"),
        X_train_fscs_rf.assign(Dataset="RF+FSCS Train"),
        X_train_srs_lgb.assign(Dataset="LGB+SRS Train"),
        X_train_fscs_lgb.assign(Dataset="LGB+FSCS Train"),
    ]
)

# Define different linestyles and widths
linestyles = {
    "Overall": "-",
    "RF+SRS Train": "--",
    "RF+FSCS Train": "-.",
    "LGB+SRS Train": "--",
    "LGB+FSCS Train": "-.",
}
linewidths = {
    "Overall": 3.5,
    "RF+SRS Train": 2.5,
    "RF+FSCS Train": 2.5,
    "LGB+SRS Train": 2.5,
    "LGB+FSCS Train": 2.5,
}

# Define colors
colors = {
    "Overall": "black",
    "RF+SRS Train": "red",
    "RF+FSCS Train": "blue",
    "LGB+SRS Train": "darkred",
    "LGB+FSCS Train": "darkblue",
}

sns.set(style="whitegrid")
# Create a 4-row, 3-column layout, one subplot for each feature
fig, axes = plt.subplots(4, 3, figsize=(25, 30))

for i, (feature, feature_name) in enumerate(zip(features, feature_names)):
    ax = axes[i // 3, i % 3]
    df_combined_data = df_combined[[feature, "Dataset"]]
    for j, (dataset, linestyle) in enumerate(
        zip(
            [
                "Overall",
                "RF+SRS Train",
                "RF+FSCS Train",
                "LGB+SRS Train",
                "LGB+FSCS Train",
            ],
            linestyles,
        )
    ):
        g = sns.kdeplot(
            data=df_combined_data[df_combined_data["Dataset"] == dataset],
            x=feature,
            label=dataset,
            ax=ax,
            linestyle=linestyles[dataset],
            linewidth=linewidths[dataset],
            color=colors[dataset],  # Specify color
        )
    ax.set_title(feature_name, fontsize=22)
    ax.set_xlabel("")
    ax.set_ylabel("Density", fontsize=20)
    ax.set_yscale("log")  # Set y-axis to log scale
    ax.tick_params(axis="both", which="major", labelsize=20)
    if i % 3 == 0:
        ax.set_ylabel("Density", fontsize=22)
    else:
        ax.set_ylabel("")

# Remove the last two empty subplots
axes[3, 1].axis("off")
axes[3, 2].axis("off")

# Place the legend outside the upper left of the first subplot
handles, labels = ax.get_legend_handles_labels()
fig.legend(
    handles,
    labels,
    loc="upper center",
    bbox_to_anchor=(0.43, 1.031),
    fontsize=21,
    title="Dataset",
    title_fontsize=21,
    ncol=5,
)

# Adjust layout
plt.tight_layout(rect=[0, 0, 0.85, 1])

plt.savefig("Ksat_Feature_Distribution_compare.png", dpi=800)

# Show the plot
plt.show()

In [ ]:
# Get the index of the 'Clay' feature
clay_idx = 1

# Create a 3x3 subplot layout, one subplot for each feature pair
fig, axes = plt.subplots(3, 3, figsize=(20, 20))

# Plot SHAP interaction dependence plots for each feature with 'Clay'
for i, feature_name in enumerate(features):
    if i >= 2:
        i = i - 1
        if feature_name != "Clay":
            row = i // 3
            col = i % 3
            shap.dependence_plot(
                (clay_idx, i + 1),
                shap_values_srs_rf_inter,
                X_train_srs_rf,
                feature_names=feature_names,
                ax=axes[row, col],
                show=False,
            )
    else:
        if feature_name != "Clay":
            row = i // 3
            col = i % 3
            shap.dependence_plot(
                (clay_idx, i),
                shap_values_srs_rf_inter,
                X_train_srs_rf,
                feature_names=feature_names,
                ax=axes[row, col],
                show=False,
            )

# Adjust subplot spacing
plt.subplots_adjust(wspace=0.21, hspace=0.12)

# Save the figure
plt.savefig("srs_rf_shap_interaction_values_clay_dependence.png", dpi=800)

# Show the plot
plt.show()

In [ ]:
# Get the index of the 'Clay' feature
clay_idx = 1

# Create a 3x3 subplot layout, one subplot for each feature pair
fig, axes = plt.subplots(3, 3, figsize=(20, 20))

# Plot SHAP interaction dependence plots for each feature with 'Clay'
for i, feature_name in enumerate(features):
    if i >= 2:
        i = i - 1
        if feature_name != "Clay":
            row = i // 3
            col = i % 3
            shap.dependence_plot(
                (clay_idx, i + 1),
                shap_values_fscs_rf_inter,
                X_train_fscs_rf,
                feature_names=feature_names,
                ax=axes[row, col],
                show=False,
            )
    else:
        if feature_name != "Clay":
            row = i // 3
            col = i % 3
            shap.dependence_plot(
                (clay_idx, i),
                shap_values_fscs_rf_inter,
                X_train_fscs_rf,
                feature_names=feature_names,
                ax=axes[row, col],
                show=False,
            )

# Adjust subplot spacing
plt.subplots_adjust(wspace=0.21, hspace=0.12)

# Save the figure
plt.savefig("fscs_rf_shap_interaction_values_clay_dependence.png", dpi=800)

# Show the plot
plt.show()

In [ ]:
# Get the index of the 'Clay' feature
clay_idx = 1

# Create a 3x3 subplot layout, one subplot for each feature pair
fig, axes = plt.subplots(3, 3, figsize=(20, 20))

# Plot SHAP interaction dependence plots for each feature with 'Clay'
for i, feature_name in enumerate(features):
    if i >= 2:
        i = i - 1
        if feature_name != "Clay":
            row = i // 3
            col = i % 3
            shap.dependence_plot(
                (clay_idx, i + 1),
                shap_values_srs_lgb_inter,
                X_train_srs_lgb,
                feature_names=feature_names,
                ax=axes[row, col],
                show=False,
            )
    else:
        if feature_name != "Clay":
            row = i // 3
            col = i % 3
            shap.dependence_plot(
                (clay_idx, i),
                shap_values_srs_lgb_inter,
                X_train_srs_lgb,
                feature_names=feature_names,
                ax=axes[row, col],
                show=False,
            )

# Adjust subplot spacing
plt.subplots_adjust(wspace=0.21, hspace=0.12)

# Save the figure
plt.savefig("srs_lgb_shap_interaction_values_clay_dependence.png", dpi=800)

# Show the plot
plt.show()

In [ ]:
# Get the index of the 'Clay' feature
clay_idx = 1

# Create a 3x3 subplot layout, one subplot for each feature pair
fig, axes = plt.subplots(3, 3, figsize=(20, 20))

# Plot SHAP interaction dependence plots for each feature with 'Clay'
for i, feature_name in enumerate(features):
    if i >= 2:
        i = i - 1
        if feature_name != "Clay":
            row = i // 3
            col = i % 3
            shap.dependence_plot(
                (clay_idx, i + 1),
                shap_values_fscs_lgb_inter,
                X_train_fscs_lgb,
                feature_names=feature_names,
                ax=axes[row, col],
                show=False,
            )
    else:
        if feature_name != "Clay":
            row = i // 3
            col = i % 3
            shap.dependence_plot(
                (clay_idx, i),
                shap_values_fscs_lgb_inter,
                X_train_fscs_lgb,
                feature_names=feature_names,
                ax=axes[row, col],
                show=False,
            )

# Adjust subplot spacing
plt.subplots_adjust(wspace=0.21, hspace=0.12)

# Save the figure
plt.savefig("fscs_lgb_shap_interaction_values_clay_dependence.png", dpi=800)

# Show the plot
plt.show()

In [ ]:
# Define covariates and target variable
features = [
    "Db",
    "Clay",
    "VFS",
    "MS",
    "OC",
    "Silt",
    "COS",
    "FS",
    "DT",
    "VCOS",
]

# Manually specify x-axis labels and units
feature_names = [
    "Db (g/cm$^3$)",
    "Clay (%)",
    "VFS (%)",
    "MS (%)",
    "OC (%)",
    "Silt (%)",
    "COS (%)",
    "FS (%)",
    "DT (cm)",
    "VCOS (%)",
]
# Get the index of the 'Clay' feature

clay_idx = 1

# Create a 3x3 subplot layout, one subplot for each feature pair

fig, axes = plt.subplots(3, 3, figsize=(20, 20))

# Plot SHAP interaction dependence plots for each feature with 'Clay'

for i, feature_name in enumerate(features):

    if i >= 2:

        i = i - 1

        if feature_name != "Clay":
            axes[row, col].grid(
                True, color="gray", linewidth=0.5, alpha=0.7
            )  # Enable grid lines

            row = i // 3

            col = i % 3

            shap.dependence_plot(
                (clay_idx, i + 1),
                shap_values_fscs_lgb_inter,
                X_train_fscs_lgb,
                feature_names=feature_names,
                ax=axes[row, col],
                show=False,
            )
            axes[row, col].set_ylabel(
                f"SHAP Interaction Value for\nClay(%) and {feature_names[i]}"
            )

    else:

        if feature_name != "Clay":
            axes[row, col].grid(
                True, color="gray", linewidth=0.5, alpha=0.7
            )  # Enable grid lines

            row = i // 3

            col = i % 3

            shap.dependence_plot(
                (clay_idx, i),
                shap_values_fscs_lgb_inter,
                X_train_fscs_lgb,
                feature_names=feature_names,
                ax=axes[row, col],
                show=False,
            )
            axes[row, col].set_ylabel(
                f"SHAP Interaction Value for\nClay(%) and {feature_names[i]}"
            )

# Adjust subplot spacing

plt.subplots_adjust(wspace=0.21, hspace=0.12)

# Save the figure

# plt.savefig("fscs_lgb_shap_interaction_values_clay_dependence.png", dpi=800) save manually

# Show the plot

plt.show()